# 📅 Day 2 — Data Cleaning & Preparation

*RTU Data Analysis & Visualization CPD course*

**📚 Instruction (3h)**  
- 🧹 Handling missing values  
- 🗑 Removing duplicates  
- 🔄 Data type conversion  
- 📅 Parsing dates  
- 🏗 Feature engineering basics  
- 🔗 Combining datasets  
- 🏷 Intro to categorical encoding  

**🛠 Practical (1h)**  
- 🧽 Clean a messy dataset  
- 🔀 Merge with a secondary dataset  

**🔄 Reflection (1h)**  
- 🧐 Review: common pitfalls in cleaning  
- 💬 Discuss real-world cleaning challenges  
- 📝 Recap exercise: identify cleaning steps for a small example dataset

## 🎯 Goals for the Day
- Strengthen Python basics (functions, loops, if/else, file handling)
- Learn to process raw messy text files into usable form
- Apply pandas methods to clean incomplete/messy data
- Merge multiple datasets into a single unified dataframe

## 💡 Motivation / Explanation

### Introduction to Data Cleaning and Preparation

- **Why cleaning is critical before analysis**
  - Raw data is almost never ready for direct analysis
  - Errors, inconsistencies, and missing information can distort results
  - Proper cleaning ensures reliability, reproducibility, and trust in analysis outcomes

- **Real-world examples of messy data**
  - 🧹 Handling missing values - Weather records with missing timestamps or corrupt values
  - 🗳️ Survey responses with inconsistent categories (e.g., "Male", "male", "M")
  - 🗑 Removing duplicates - Financial transactions with duplicate entries
  - 🗑 Log files with noise lines, system messages, or broken encodings
  - 📅 Parsing dates - Event logs with inconsistent timestamp formats
  - 🔄 Data type conversion - User age recorded as text instead of numbers

> Think of data cleaning as *“washing vegetables before cooking”* — not exciting, but essential for a good meal.


## Weather Dataset: `latvia_meteo_1925_messy.zip`

Let us imagine we are helping Toms Bricis with weather data analysis for the year 1925. We have come across a bundle of messy text files that require cleaning and preparation.

- **What it is:** a bundle of **five “messy” text files** (≈50 rows each) simulating daily measurements from Latvian stations in **1925**:
  - **Rīga-University** — *Period 1 (Jan–Mar)*
  - **Rīga-University** — *Period 2 (Sep–Nov)*
  - **Liepāja** — *Apr–Jun*
  - **Mērsrags** — *Feb–May*
  - **Alūksne** — *Oct–Dec*

- **Columns present (but order varies by file):**  
  `date`, `t_max_c`, `t_min_c`, `precip_24h_mm`, `precip_type`, `present_weather_code`, `notes`

- **Deliberate “messiness” to practice cleaning:**
  - **Different separators:** `;`, `,`, `|`, and **TAB** (documented in each file’s `# fields=` header).
  - **Mixed column order** across files (use the header to map columns).
  - **Date formats vary** (`YYYY-MM-DD`, `DD.MM.YYYY`, `YYYY/MM/DD`, `DD-MM-YYYY`, `MM-DD-YYYY`, `YYYY.MM.DD`) and sometimes **include a time** (e.g., `07:00`).
  - **Numeric quirks:** decimal **commas** (e.g., `0,6`), **units** in strings (e.g., `0.8 mm`), and the Latvian word **“nulle”** for zero.
  - **Missing values** sprinkled in as `""`, `NA`, `—`, `-999`.
  - **Codes as strings** with possible leading zeros (e.g., `present_weather_code = "05"`).
  - **Free-text `notes`** in Latvian from the station master (may be blank/missing).

- **Intended skills to practice (Day 2):**
  - Detect & use **separators/column order** from headers.
  - **Parse heterogeneous dates** (with optional times).
  - Normalize **numerics/units** (decimal commas, `mm`, worded zeros).
  - Unify **missing values** and enforce **types** (e.g., cast weather codes to integers).
  - Keep useful **categorical text** (`precip_type`, `notes`) intact.

As part of our workflow we will want to verify whether the above descriptions of messiness hold true for our specific dataset files. This will help us tailor our cleaning approach effectively.


## Part 1: Python Fundamentals for Data Cleaning

For our first part we will use basic Python programming skills to explore and understand the dataset structure before diving into the cleaning process.

### 🔑 Key Idea - Loops Go Brrr

One of key advantages of programming is that we can automate repetitive tasks using loops. This is especially useful when working with datasets, as it allows us to apply the same operations to multiple rows or files without having to write redundant code.

Similarly loops let us figure out an approach that works for a single file and then easily adapt it to others.

### 🔑 Key Idea - Branching

Branching (using `if`, `elif`, and `else` statements) allows your code to make decisions and execute different blocks of code based on whether certain conditions are true or false. This is fundamental for creating programs that can respond to varying inputs or data characteristics.

### 🔑 Key Idea - Functions

Functions are reusable blocks of code that perform a specific task. They help organize code, avoid repetition, and make programs easier to understand and maintain.

### Getting ready for work

Typically in a finished notebook (and also normal scripts / programs), we want to start with a clear setup phase. This includes:

1. **Importing Libraries:** Load all necessary libraries at the beginning.
2. **Setting Up Paths:** Define file paths and other constants.
3. **Configuring Options:** Set any options or preferences (e.g., display settings).

By organizing our code this way, we make it easier to understand and modify later on.


In [1]:
# usually we start with general Python imports

import sys, platform, os, io, shutil, zipfile, re # system related tasks - this is full syntax we import full modules
# below with from some_module import some sub_object
from pathlib import Path # for file and file path related tasks
from datetime import datetime # this is because datetime module has datetime class
# otherwise if we used import datetime
# we would have to write datetime.datetime for everything
# first datetime
print(f"Today : {datetime.now().isoformat(timespec='seconds')}")
# now Python version
print(f"Python : {sys.version}")

# then we import external libraries
# external - not part of Python installation
# on Google Colab those are already installed
try:
    # note how in Python after colon we have indentation usually
    import pandas as pd # we could fail at import, then next line would not run
    print('pandas:', pd.__version__)
except ImportError:
    print(f"pandas not installed. Install with `pip install pandas`.")
    # for excel support extra instructions
    print(f"Install `openpyxl` for Excel support with `pip install openpyxl`.")
# requests is a widely used network library that makes internet "requests" easier
try:
    import requests # popular web requests library, for scraping, downloading web resources etc
except ImportError:
    requests = None
    print('requests not installed. Install with `pip install requests`.')


# we can also print out what type of environment we are running in, this could show OS information
print('Runtime:', platform.platform())
# We could show system RAM and free RAM but that would require either a non standard library
# or we would have to write some extra functions we skip this for now
# you can ask LLM to write these functions for you
# alternatively there are external libraries like psutil that do this out of the box

# Let us show our current drive space
print(f"Total Current Drive Space: {shutil.disk_usage('/').total / (1024**3):.2f} GB")
print(f"Free Current Drive Space: {shutil.disk_usage('/').free / (1024**3):.2f} GB")

# Current Working Directory
print(f"Current Working Directory: {Path.cwd()}") # more important in local computer
# in Google Colab this should give you /content
# note that in some cases you might not want to provide all this information to the public, if you have a super secret computer...


Today : 2025-10-27T16:17:09
Python : 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
pandas: 2.2.2
Runtime: Linux-6.6.105+-x86_64-with-glibc2.35
Total Current Drive Space: 107.72 GB
Free Current Drive Space: 68.04 GB
Current Working Directory: /content


In [3]:
# let's also get stats on total, free and used memory
# we can use psutils which is another external library
try:
    import psutil
    print('psutil:', psutil.__version__)
    # print memory usage
    print(f"Total Memory: {psutil.virtual_memory().total / (1024**3):.2f} GB")
    print(f"Available Memory: {psutil.virtual_memory().available / (1024**3):.2f} GB")
except ImportError:
    print(f"psutil not installed. Install with `pip install psutil`.")



psutil: 5.9.5
Total Memory: 12.67 GB
Available Memory: 11.50 GB



### Branching with if



In [6]:
# simple decision we make one of choice only
# just like in https://www.poetryfoundation.org/poems/44272/the-road-not-taken
is_raining = True # new Boolean variable True or False, could be getting data from some source
is_raining = False # so I am overwritting previous value if it exists
if is_raining:  # note how there is indentation after colon in Python for many operations we have separate paths now
    print("Bring an umbrella! ☂")
    print("Prepare for heavy rain!")
    # I could do more here for True branch
else: # means is_raining is False - else branch is not required
    print("No umbrella needed. ")
    # show sun emoji
    print("🌞")
    # I could do more here for False branch
    # we could have had sub braches as well

# here we are out of the if branches
print("Back to usual stuff") # this always runs

No umbrella needed. 
🌞
Back to usual stuff


In [9]:
# we can compare something when making a decision
# a = 105
a = 20
b = 20
if a > b: # this check will be first
    print(f"{a} is greater than {b}")
elif a < b: # this check will be second
    print(f"{a} is less than {b}")
else: # so when other options are exhausted
    print(f"{a} is equal to {b}")

20 is equal to 20


In [12]:
# we can check if values are equal with == not = which is assignment!
if a == b:
    print(f"{a} is equal to {b}")

20 is equal to 20


### Loops

### 🔄 For Loops

For loops in Python let us iterate over a sequence (like a list, tuple, or string, or other iterables such as lines in a file) and perform an action for each item in that sequence.

General syntax of for loops is:

```Python
for element in <iterable>:
    <action>
    <more optional action>
```

Note the indentation, as usual in Python after : we have indentation to indicate the block of code that belongs to the for loop.

In [13]:
my_name = "Valdis"
# if you think about it a string is a collection of letters / small strings
# let's loop over characters one item at a time
for char in my_name: # char is just a variable name I made up
    print(char)

V
a
l
d
i
s


In [14]:
# we can use other iterables such as lists
shopping_list = ["apples", "bananas", "milk", "bread"]
for item in shopping_list: # item is a new variable that changes each time we iterate/loop over collection
    print(f"Need to buy {item}")
    # still inside loop

# outside loop

print("All done with shopping")

Need to buy apples
Need to buy bananas
Need to buy milk
Need to buy bread
All done with shopping


In [15]:
# we can also iterate over some range
# note how range(5) actually iterates over 0 to 4 included
for num in range(5): # instead of num I could use x or i or whatever
    print(f"The number currently processed is {num}")

The number currently processed is 0
The number currently processed is 1
The number currently processed is 2
The number currently processed is 3
The number currently processed is 4


In [18]:
# if you need custom range let's say 100 to 120 with 2 step
# so full syntax for range is: range(start, stop_not_included, step)
for num in range(100, 121, 5):
    print(num)

100
105
110
115
120


### 🧑‍💻 Functions

### What is the idea behind functions in programming?

In programming, a function is a block of reusable code that performs a specific task. Think of it like a miniature program within your main program. Functions are designed to:

- **Break down complex problems:** Large problems can be divided into smaller, manageable parts, each handled by a function. This makes code easier to write, understand, and debug.
- **Avoid repetition (DRY principle - Don't Repeat Yourself):** If you need to perform the same set of actions multiple times, you can define a function once and call it whenever needed, rather than writing the same code repeatedly.
- **Improve code organization and readability:** Functions group related code together, making the overall structure of your program clearer and easier to follow.
- **Enhance code reusability:** Once a function is defined, it can be used in different parts of the same program or even in other programs.
- **Simplify debugging:** If there's an issue, you can isolate the problem to a specific function, making it easier to find and fix the error.

In essence, functions are tools for modularity and abstraction in programming, allowing you to create more organized, efficient, and maintainable code.

### What is a function in Python?

In Python, a function is defined using the `def` keyword, followed by the function name, parentheses `()`, and a colon `:`. The code block within the function is indented. Functions can optionally take inputs called *arguments* (placed inside the parentheses) and can return a value using the `return` keyword.

Here's a basic structure of a Python function:

In [19]:
# super simple function
def simple_greet(): # so again using snake_case for multiple word names
    print("Hi there!")
    print("Let's do some work!")
    # function ends here


In [20]:
# calling simple_greet()
simple_greet()

Hi there!
Let's do some work!


In [21]:
for _ in range(3): # _ is convention that variable is not needed
  simple_greet()

Hi there!
Let's do some work!
Hi there!
Let's do some work!
Hi there!
Let's do some work!


In [23]:
# let's write a function that greets someone n times
def greet_many(n):
  # so we are inside our function and n has just come in
  for _ in range(n): # again _ represent a variable we do not care about
    simple_greet()

In [24]:
# now i can greet someone 5 times
greet_many(5)

Hi there!
Let's do some work!
Hi there!
Let's do some work!
Hi there!
Let's do some work!
Hi there!
Let's do some work!
Hi there!
Let's do some work!


In [25]:
# above could be useful but it is much more useful if functions take parameters/arguments and return something (data)
# let's have a super simple function create_greeting
def create_greeting(name):
  # we do nothing inside the body of this function we just returned a transformed value
    return f"This is a custom greeting for {name}"

# here we are not going to be printing anything instead we will be returning something so we can reuse



In [26]:
my_greeting = create_greeting("Valdis")
print(my_greeting)

This is a custom greeting for Valdis


In [27]:
ede_greeting = create_greeting("Ede")
print(ede_greeting)

This is a custom greeting for Ede


In [28]:
# full syntax
def greet(name: str = 'student') -> str:
    """
    Returns a friendly greeting for the given name.
    If no name is provided, defaults to 'student'.
    """
    # """ is so called Docstring a way in Python to include small help snippets
    # name is a variable local to this function
    # Use an f-string to insert the name into the greeting
    # note we are not printing the greeting here, just returning it for use by another part of the code
    return f"Hello, {name}!" # we can insert pretty much any type of data in f-strings

# also nothing should be happening besides the function being added to our memory
# we have not called the function yet

In [29]:
# Call the function and print the result
greeting = greet() # assign results of greet() function to greeting variable
print(greeting)

Hello, student!


In [31]:
# Now that I have this function I can make other greetings
my_greeting = greet("Valdis")
print(my_greeting)
numeric_greeting = greet(80800) # function expects str, but no penalty in this case
print(numeric_greeting)

Hello, Valdis!
Hello, 80800!


### Type Hints

Type hints in Python provide a way to indicate the expected data types of variables, function parameters, and return values. They help improve code readability and enable better static analysis by tools like linters and IDEs.

However, they have no actual power at runtime and are not enforced by the Python interpreter. They serve as a guideline for developers and can be checked using static type checkers like mypy.

I liken them to "documentation for your code." Just as documentation helps users understand how to use your code, type hints help developers understand what types of values are expected.



In [32]:
# let's see one more example with simple add function
def add(a: int, b: int) -> int:
    return a + b

# so this function expects only integers and returns integer

# however it will work with any values that support adding



In [33]:
print(add(2,2))
print(add(3.14, 2.71))
print(add("Valdis", " RTU"))

# so again type hints are useful (AI will happily make them for us)
# but in Python they only serve as a guiding light

4
5.85
Valdis RTU


In [35]:
# we can still get error if we try to add str with integer which is not allowed in Pythong...
# print(add("nevar", 66))
print(add("nevar", str(66)))  # i type-cast 66 to string

nevar66


In [37]:
# let's make a multiplication function that takes integer and string and returns a string
def multiply(a: int, b: str = "x") -> str:
    return a * b

print(multiply(2,3))
print(multiply(3.1415926, 44444))
print(multiply("Beer ", 5)) # funnily Python lets us multiply Strings with integers
print("*"*80) # i pass * explicitly
print(multiply(20)) # here x is implicit as value for b because it is default

6
139624.9415144
Beer Beer Beer Beer Beer 
********************************************************************************
xxxxxxxxxxxxxxxxxxxx


### Function to download and unzip file from url

Below is a more complicated function that does two things: it downloads a zip file from a given URL and then extracts its contents to a specified directory.

Theoretically speaking it would be better to have two functions one that downloads the file and another that extracts it.

In general functions should do one thing and do it well.

When you need to do more than one thing you can combine them into a single function, but be mindful of keeping the function focused and not overly complex.


In [38]:
text = "A quick brown fox     jumped over a sleeping dog"
words = text.split() # split creates a list of strings from string
# by default it splits by any whitespace including newlines \n, tabs \t, and of course spaces
print(words)
# we can split by other symbol or symbols
comma_text = "Valdis, Līga, Maija, Rūta, Ede"
comma_words = comma_text.split(",") # i could have supplied any other divider
print(comma_words)
# if I did not want the extra space I could have split by ", "
comma_words_clean = comma_text.split(", ")
print(comma_words_clean)

['A', 'quick', 'brown', 'fox', 'jumped', 'over', 'a', 'sleeping', 'dog']
['Valdis', ' Līga', ' Maija', ' Rūta', ' Ede']
['Valdis', 'Līga', 'Maija', 'Rūta', 'Ede']


In [39]:
# two ways of accessing values by index in Python
# we have positive index starting with 0
# we have negative index starting with -1 and counting backwards
print("First", comma_words_clean[0])
print("Second", comma_words_clean[1])
print("Last", comma_words_clean[-1])
print("Second to last", comma_words_clean[-2])
print("Third", comma_words_clean[2])
print("Third from the end", comma_words_clean[-3])

First Valdis
Second Līga
Last Ede
Second to last Rūta
Third Maija
Third from the end Maija


In [40]:
def download_and_unzip(url: str, target_folder: str | Path = 'sample_data') -> Path:
    """
    Downloads a ZIP file from the given URL and extracts it to the target folder.
    Returns the path to the folder where files were extracted.
    """
    target = Path(target_folder)  # Make sure target is a Path object
    # Path makes it so all 3 major OC (Mac, Windows, Linux) work similarly
    target.mkdir(parents=True, exist_ok=True)  # Create the folder if it doesn't exist
    filename = url.split('/')[-1]  # Get the file name from the URL
    # in case of URL the file name is the last one (so index -1 means last one in a list)
    # next we create full path where we will save the zip file
    zip_path = target / filename  # Full path to save the ZIP file
    print(f"Saving to {zip_path}")
    # check if library exists
    if requests is None:
        raise RuntimeError('requests required.')
    # Download the file in chunks (good for large files)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()  # Raise an error if download failed
        with open(zip_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    # Unzip the downloaded file
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(target) # this command given a zip file as target will unzip ALL files, we could use pwd parameter if password is needed to decrypt
    # Optionally, remove the ZIP file after extraction
    # zip_path.unlink(missing_ok=True)  # Remove the ZIP file similar to moving to recycle bin
    return target  # Return the folder where files were extracted


**Practice dataset for Part 1:** `latvia_meteo_1925_messy.zip` (5 text files)

- URL: https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip

In [41]:
# let's download the messy files zip
url = "https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip"
print(f"Will download and unzip from following url: {url}")
# let's download the file and extract it under day_2_data
download_and_unzip(url, Path("day_2_data"))

Will download and unzip from following url: https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip
Saving to day_2_data/latvia_meteo_1925_messy.zip


PosixPath('day_2_data')

## Exploring Folder content

In [42]:
text_files = sorted(Path("day_2_data").glob("*.txt")) # so I look for all files that end with txt extension
print(text_files)
# so glob is kind of like file search in Windows Explorere
for p in text_files: # so we iterate/loop over all file paths, calling each one with name p - i could have used a longer more descriptive name such as file_name, my_file, file etc etc
    print(p)

[PosixPath('day_2_data/aluksne_1925.txt'), PosixPath('day_2_data/liepaja_1925.txt'), PosixPath('day_2_data/mersrags_1925.txt'), PosixPath('day_2_data/riga_university_1925_p1.txt'), PosixPath('day_2_data/riga_university_1925_p2.txt')]
day_2_data/aluksne_1925.txt
day_2_data/liepaja_1925.txt
day_2_data/mersrags_1925.txt
day_2_data/riga_university_1925_p1.txt
day_2_data/riga_university_1925_p2.txt


In [43]:
# let's add Google Colab specific code that will offer download to your computer of all the files extracted
# this will not work locally since you already have these files locally!
from google.colab import files # this is Google Colab specific
# if you were local you would already have these files locally
# so we will loop through all files with txt extension in day_2_data and download them
for p in sorted(Path("day_2_data").glob("*.txt")): # glob looks in current folder
    files.download(p) # so this command will run for each file path - we call it p here

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 📂 File Handling

First let me show you how to read whole text file into one big text string

In [44]:
# let's make a function that takes a file Path or string and optional encoding with default utf-8 and returns text string
def get_file_contents(path: Path | str, encoding: str = 'utf-8') -> str:
    """
    Reads the contents of a text file and returns it as a string.
    """
    # so path could be string or Path
    # file could have any extension but it should contain text of some sort
    with open(path, mode='r', encoding=encoding) as f:
        # file stream is open here under f variable which has type IOStream(sic)
        content = f.read() # so we came up with name content for our inner variable
        # file is still open even though we have moved the reading "head" to the end
    # here file is automatically closed after the with block
    return content

In [45]:
# let's read aluksne into memory
# aluksne = get_file_contents(Path("day_2_data/aluksne_1925.txt"))
aluksne = get_file_contents("day_2_data/aluksne_1925.txt")
print(aluksne[:300]) # let's print first 300 characters
# note above is same as print(aluksne[0:300]) # 0 is default

# station_name=Alūksne
# period=(Oct–Dec)
# separator_hint=|
# columns_in_this_file= notes | present_weather_code | t_max_c | precip_24h_mm | date | precip_type | t_min_c
# note: some values intentionally messy (units, words, missing, time in date)
# fields=notes|present_weather_code|t_max_c|precip_


In [46]:
# print last 200 characters
print(aluksne[-200:]) # Python offers two types of indexing
# positive from 0 to len(iterable)-1
# negative from -len(iterable) to -1

0.2 mm|12-18-1925|snow|-7.6
Pūtis brāzmas|82|5.3||11-18-1925||-0.4
NA|63|3.4||12-19-1925|M|-6.0
Novērojums vēlāk|90|-0.2|2.3|12-22-1925|mixed|-8.2
Daļēji mākoņains|90|-3.5|0.2 mm|12-16-1925|rain|-4.0



![Python indexing](https://developers.google.com/static/edu/python/images/hello.png)

## Simple string methods

In [47]:
# we can get count of word "brāzmas" in content
aluksne.count("brāzmas") # count is a string method

4

In [48]:
# we could ask if rain is in aluksne
# this is so called existence check
"rain" in aluksne # so ancient Aluksnians wrote in English... :)

True

In [49]:
# we could replace some text in this content
# however strings are immutable - unchangable
# so to change we would have to overwrite or have new variable
# let's have new text with replacement of Mērījums apstiprināts to Viss OK
aluksne_ok = aluksne.replace("Daļēji mākoņains", "Pavisam drūmi")
# Note replace DOES NOT modify original text
# we have to supply a new variable to save this changed text
# if I wanted aluksne modified I would have written as follows
# aluksne = aluksne.replace("Mērījums apstiprināts", "Viss OK")
print(aluksne_ok)

# station_name=Alūksne
# period=(Oct–Dec)
# separator_hint=|
# columns_in_this_file= notes | present_weather_code | t_max_c | precip_24h_mm | date | precip_type | t_min_c
# note: some values intentionally messy (units, words, missing, time in date)
# fields=notes|present_weather_code|t_max_c|precip_24h_mm|date|precip_type|t_min_c
—|53|5.9|3.0 mm|10-19-1925|R|5.5
Rīts auksts|-999|9.1|2.3 mm|18.10.1925|mixed|4.2
Pavisam drūmi|65|5.4|0.0|11-10-1925||-0.5
Pavisam drūmi|50|15.6|0.5|10-07-1925 14:00|S|10.4
Neliels vējš|63|-1.1|0.0|12-28-1925|—|—
Vēlāk sāka līt|-999|5.3||12-07-1925 19:00|none|
Pūtis brāzmas|82|4.2|0.6 mm|11-07-1925|S|0.1
—|61|-7.2|1.4|12-03-1925|mixed|-6.5
Laikam migla|51|3.3|1.0 mm|11-27-1925|mixed|-1.3
Mērīšanas kļūda?|70|-3.5999999999999996|2.1|12-06-1925|M|-2.9
Skursteņi kūp|71|6.2|0.0|10-21-1925||6.9
NA|61|1.3|2.1|28.11.1925|NA|2.0
Ap pusdienlaiku saule|80|8.4|1.9 mm|02-11-1925|snow|-1.5
|53|8.4||10-01-1925 07:00||3.3
Mērījums apstiprināts|53|10.8|0.7 mm|10-08-1925|S|5.3

### Iterating over file line at a time

Above example showed how we could read a whole file into memory.
However that means we would be working with file as one big string as a whole. We could do some replace operations.

Much more often we will want to work on file one line(row) at a time.

In [50]:
def count_lines(path: Path | str) -> int:
    """
    Counts the total number of lines and the number of non-empty lines in a file.
    Returns a tuple: (total_lines, nonempty_lines)
    """
    total = 0 # we start our counter with 0, very common pattern in software development
    # this total is local to the count_lines function
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        # here file is open
        for line in f: # so f is a filestream with this for loop we go through it line at a time
            total += 1 # remember this is same as total = total + 1
            # we could do something more here with the line but here we just count the contents
    # important that file is here closed automatically
    return total # Return the line count

In [51]:
# let's count lines in aluksne_1925
count_lines(Path("day_2_data/aluksne_1925.txt"))

42

In [52]:
# Path is not strictly necessary , Path is used to avoid cross OS errors between Mac,Windows, Linux
count_lines("day_2_data/aluksne_1925.txt")

42

In [53]:
# let's count mersrags
count_lines("day_2_data/mersrags_1925.txt")

54

In [54]:
# let's count lines for all *.txt files in day_2_data
for p in sorted(Path("day_2_data").glob("*.txt")):
    print(f"{p.stem} has {count_lines(p)} lines")

aluksne_1925 has 42 lines
liepaja_1925 has 42 lines
mersrags_1925 has 54 lines
riga_university_1925_p1 has 42 lines
riga_university_1925_p2 has 42 lines


In [55]:
# let's write a function that takes a file path for text file
# second paramater will be destination folder where to write cleaned file
# we want to go line by line and save only lines that starts_with # as default
def clean_file(file, dst, prefix = "#", needle="", postfix="", encoding="utf-8"):
    # first let's convert file to Path just in case
    file = Path(file)
    with open(file, mode='r', encoding=encoding, errors='replace') as f:
      # let's create dst folder if it does not exist
      dst = Path(dst) # just in case
      # now we can make folder
      dst.mkdir(exist_ok=True) # if folder exists do nothing
      # let's define save file as dst plus file.stem plus cleaned plus .txt
      dst_file = dst / (file.stem + "_cleaned.txt") # dst is Path object
      with open(dst_file, mode='w', encoding=encoding, errors='replace') as g: # this file does not have to exist, it will be created automatically
            for line in f: # so we go row by row in original file
              if line.startswith(prefix) and needle in line and line.endswith(postfix): # so prefix could be anything you want
              # you can adjust logic in above line to whatever fits your use case
                g.write(line)


In [56]:
# so let's try saving some data from aluksne_1925.txt
clean_file("day_2_data/aluksne_1925.txt", "day_2_cleaned")

In [57]:
# let's save all lines where Rīts is present
# note I have to pass prefix as "" because Rīts is present on lines where # is not beginning
clean_file("day_2_data/aluksne_1925.txt", "day_2_morning", prefix="", needle="Rīts")

## Determining the separator character

Our files have helpfully provided a separator hint without which it would be very hard to determine.

`# separator_hint=|`
`# separator_hint=TAB`
`# separator_hint=,`

Let's write a function that extracts hint from the file


In [58]:
# "Alice Bob     Carol   ".split("Bob")[-1]
"Alice Bob     Carol   ".split("Bob")[-1].strip()
# strip removes all whitespace characters from left and right side
# there is also rstrip() and lstrip() if needed

'Carol'

In [ ]:
# let's use regular expression to extract any letters after Bob
# in string such as "Alice Bob     Carol   " I want to return Carol only
# import re # regular expressions
# haystack = "Alice Bob     Carol   "
# needle = "Bob"
# my_match = re.search(needle, haystack)
# print(my_match)
# TODO find a matching group

In [59]:
def get_sep(path: Path) -> str:
    """
    We look for first line that contains
    `# separator_hint=|`
    `# separator_hint=TAB`
    `# separator_hint=,`
    """
    needle = "separator_hint="
    sep = None # we start with assumption that we do not know the separator
    # lets open file and go through line by line
    with open(path, 'r', encoding='utf-8', errors='replace') as f:
        for line in f: # so go row by row
            if needle in line: # we check for presence of needle
            # we could have used regular expression but no need here
                # let's find whatever is after needle
                # so we split by needle at take last part
                sep = line.split(needle)[-1].strip() # strip removes all whitespace from both sides of string
    # file is automatically closed here
    # if sep is TAB we need to return \t
    if sep == "TAB": # special case TAB which we need to convert
        return "\t" # so \t is a way of representing tab in Python and many other languages
        # similarly \n is a newline symbol
        # so then \\ is \ symbol
    return sep

In [60]:
# let's see if we can find separator for aluksne_1925
get_sep(Path("day_2_data/aluksne_1925.txt"))

'|'

In [61]:
# let's find it for all txt files
for file_name in sorted(Path("day_2_data").glob("*.txt")):
    print(file_name, get_sep(file_name)) # so we call get_sep for each file

day_2_data/aluksne_1925.txt |
day_2_data/liepaja_1925.txt ,
day_2_data/mersrags_1925.txt 	
day_2_data/riga_university_1925_p1.txt ;
day_2_data/riga_university_1925_p2.txt ;


## glob vs. rglob

Using Path I can use glob or rglob to find files matching my pattern

glob looks in specific directory only

rglob looks in specific directory and ALL subfolders and their subfolders and so on forever

rglob is quite similar to how Windows Explorer looks for files

In [62]:
# let's use rglob to look for all text files in current folder and its subfolders and so on
text_files = sorted(Path(".").rglob("*.txt")) # . means current folder
# rglob actually returns and iterator not list
# sorted returns a sorted by name list
for p in text_files:
    print(p)

day_2_cleaned/aluksne_1925_cleaned.txt
day_2_data/aluksne_1925.txt
day_2_data/liepaja_1925.txt
day_2_data/mersrags_1925.txt
day_2_data/riga_university_1925_p1.txt
day_2_data/riga_university_1925_p2.txt
day_2_morning/aluksne_1925_cleaned.txt


In [63]:
# now let's find all csv files
csv_files = sorted(Path(".").rglob("*.csv"))
for p in csv_files:
    print(p)

sample_data/california_housing_test.csv
sample_data/california_housing_train.csv
sample_data/mnist_test.csv
sample_data/mnist_train_small.csv


In [64]:
# note difference if we gave the actual subfolder
txt_files = sorted(Path("day_2_data").rglob("*.txt"))
# no difference between rglob or glob because no subfolders for day_2_data
for p in txt_files:
    print(p)

day_2_data/aluksne_1925.txt
day_2_data/liepaja_1925.txt
day_2_data/mersrags_1925.txt
day_2_data/riga_university_1925_p1.txt
day_2_data/riga_university_1925_p2.txt


## Hand loading a DataFrame

For single cases we could do okay by hand, meaning we use
Pandas read_csv and apply some parameters to skip some rows, use custom separator etc.

In [65]:
# so let's load aluksne
# we know the following it uses | as separator
# first 6 rows contain metadata that we want to skip
# we want column names to be generic
aluksne_df = pd.read_csv(Path("day_2_data/aluksne_1925.txt"),
                         sep="|",
                         skiprows=6,
                         header=None)
aluksne_df.head()

,0,1,2,3,4,5,6
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—


## Supported Text Encoding by Python

Usually we want utf-8, but you might have some older text files that you might want to process.

* https://docs.python.org/3/library/codecs.html#text-encodings

In [66]:
# now let's read row 6 from aluksne_1925.txt, it contains needle "fields="
# we want to split by this needle
# then the last part we want to split by pipe |
# those will be the column names
with open(Path("day_2_data/aluksne_1925.txt"), 'r', encoding='utf-8', errors='replace') as f:
    for line in f:# again loop through each line one line at a time
        if "fields=" in line:
            column_names = line.split("fields=")[-1].strip().split("|") # here we know the separator | but in general we need to find sep first
            break # we assume there will be no more fields= we already got our column_names

column_names

['notes',
 'present_weather_code',
 't_max_c',
 'precip_24h_mm',
 'date',
 'precip_type',
 't_min_c']

In [67]:
def get_column_names(path: Path | str, needle: str = "fields=", sep: str = "|") -> list[str]:
    """
    Extracts column names from a file by looking for a line containing the needle
    and splitting the part after the needle by the specified separator.
    """
    columns = [] # standard approach we start with nothing - empty list
    with open(path, mode='r', encoding='utf-8', errors='replace') as f:
        for line in f:
            if needle in line:
                # Split by needle and take the last part, then strip whitespace
                raw_fields = line.split(needle)[-1].strip()
                # Split the raw fields by the separator
                columns = raw_fields.split(sep)
                break  # Assuming only one line with the needle
    return columns

In [68]:
# let's extract columns from aluksne using function
column_names = get_column_names(Path("day_2_data/aluksne_1925.txt"))
column_names

['notes',
 'present_weather_code',
 't_max_c',
 'precip_24h_mm',
 'date',
 'precip_type',
 't_min_c']

In [71]:
# let print columns for some other file
# in this case I am NOT saving columns just printing
# print(get_column_names(Path("day_2_data/mersrags_1925.txt")))
print(get_column_names(Path("day_2_data/mersrags_1925.txt"), sep="\t"))

['date', 'precip_24h_mm', 'precip_type', 'notes', 't_min_c', 'present_weather_code', 't_max_c']


In [72]:
# let's check length of df columns and how many column_names we have
print(len(aluksne_df.columns))
print(len(column_names))
# print ready for assignment message if lengths match
if len(aluksne_df.columns) == len(column_names):
    print("Lengths match") # we are good to go here!
    # means we can assign columns here to df
else:
    print("Lengths do not match") # we need to think why thye did not match

7
7
Lengths match


In [73]:
# now let's apply those column names to aluksne_df
# only requirement that lengths of columns match,
# that is we have same number of columns
aluksne_df.columns = column_names # so column names had matching length
aluksne_df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—


### 📥 Loading Cleaned Files into DataFrames

In [74]:
# Let us write a function that will take a file path
# This function will go through file line by line
# If line with separator_hint= is found we save in sep value the last part of this line after stripping whitespace
# any other line that starts with hash (#) is ignored
# all other lines are split using sep value and stored as a list of lists (2d)
# once all lines are read the 2d list is converted to Dataframe and returned
def load_messy_file(path: Path | str) -> pd.DataFrame:
    sep = None
    lines = [] # for storing 2D list of elements
    # we will use lines to pass later to DataFrame constructor
    columns = []
    # with guarantees closing of file at the end
    with open(path, mode='r', encoding='utf-8', errors='replace') as f:
        # we loop through file one row at a time
        for line in f:
            if line.startswith('# separator_hint='):
                # extract separator by splitting by hint and then taking last item
                sep = line.split('separator_hint=')[-1].strip()
                # add special case TAB -> \t
                if sep == "TAB":
                    sep = "\t"
            if "# fields=" in line: # this function only works correctly
              # if we get sep before fields= row
              # we split by "fields=" then split last part by sep giving us columns
              raw_fields = line.split("fields=")[-1].strip() # we take last part
              columns = raw_fields.split(sep) # if we split by None we will get error
            if line.startswith("#"): # order is important we check this AFTER hint check
                continue # we go to next line
            # this means here we have a good line that should be usable for data
            if sep:
                # we split the row by separator and add it to our list of lines
                lines.append(line.split(sep)) # we create a new list
                # and append this new list to our existing
                # in effect we are creating a 2D matrix - 2d list here
    # Here file is closed - that's good and safe!
    # all that remains is to create a dataframe from our lines
    df = pd.DataFrame(lines) # we leave the challenge of creating appropriate names for later
    # let's check if column count in df corresponds to length of our columns
    if len(columns) == len(df.columns):
        # if it matches we apply our columns
        df.columns = columns
    # so above idea is to use our columns only when our shape matches
    return df

In [75]:
# let's try above function with aluksne_1925.txt
df = load_messy_file(Path("day_2_data/aluksne_1925.txt"))
df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53,5.9,3.0 mm,10-19-1925,R,5.5\n
1,Rīts auksts,-999,9.1,2.3 mm,18.10.1925,mixed,4.2\n
2,Daļēji mākoņains,65,5.4,0.0,11-10-1925,,-0.5\n
3,Daļēji mākoņains,50,15.6,0.5,10-07-1925 14:00,S,10.4\n
4,Neliels vējš,63,-1.1,0.0,12-28-1925,—,—\n


In [76]:
text_files = sorted(Path("day_2_data").glob("*.txt"))
text_files

[PosixPath('day_2_data/aluksne_1925.txt'),
 PosixPath('day_2_data/liepaja_1925.txt'),
 PosixPath('day_2_data/mersrags_1925.txt'),
 PosixPath('day_2_data/riga_university_1925_p1.txt'),
 PosixPath('day_2_data/riga_university_1925_p2.txt')]

In [77]:
df_dictionary = {} # dictionary is a data structure in Python
# other languages call this hashtable, associative array
# idea is that from unique key we can find very quickly the value
# which uses key -> value, so unique keys point to any type of value
# typically keys are strings, but technically any hashable key can be used
for file in text_files:
    df_dictionary[file.stem] = load_messy_file(file) # so we assign to each stem of file name a DataFrame created from that file
# how many dataframes
print(len(df_dictionary))

5


In [78]:
# let's go through all dataframes, print the file name and display head(3)
for key, value in df_dictionary.items(): # items gives us iterable tuples of key and value commonly we use k,v for short
# here key == file_name stem (no extension)
# value == dataframe
    print("Data source", key)
    display(value.head(3)) # show first 3 rows

Data source aluksne_1925


,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53,5.9,3.0 mm,10-19-1925,R,5.5\n
1,Rīts auksts,-999,9.1,2.3 mm,18.10.1925,mixed,4.2\n
2,Daļēji mākoņains,65,5.4,0.0,11-10-1925,,-0.5\n


Data source liepaja_1925


,0,1,2,3,4,5,6,7,8,9
0,13,2,1925/05/26,none,75,0,0 mm,-999,Mērīšanas kļūda?\n,None
1,24,4,07.06.1925,rain,75,0,6,16,8,Neliels vējš\n
2,12,600000000000001,1925/05/20,S,50,0,4 mm,13,3,Pūtis brāzmas\n


Data source mersrags_1925


,date,precip_24h_mm,precip_type,notes,t_min_c,present_weather_code,t_max_c
0,15-03-1925,0.0,none,Mērījums apstiprināts,-3.7,70,-4.4\n
1,04-04-1925 14:00,0.6,snow,Ap pusdienlaiku saule,4.7,75,7.9\n
2,22-05-1925,0.1,,,7.7,80,11.9\n


Data source riga_university_1925_p1


,date,t_min_c,t_max_c,present_weather_code,precip_type,precip_24h_mm,notes
0,1925-02-11,—,1.3,73,,0.0,—\n
1,1925-01-06,-4.7,0.8,90,none,0.1 mm,Mērīšanas kļūda?\n
2,1925-02-07,-8.3,-7.2,71,none,0.0 mm,Rīts auksts\n


Data source riga_university_1925_p2


,present_weather_code,date,precip_24h_mm,t_max_c,t_min_c,precip_type,notes
0,53,18.10.1925 20:00,0.7,10.5,6.0,snow,—\n
1,,1925/10/13,0.0,9.7,8.1,—,\n
2,50,16.09.1925,1.3,17.3,14.8,rain,Mērīšanas kļūda?\n


In [79]:
# okay let's write a function that takes src folder containing text files
# function takes also dst folder for output of xlsx files that will be obtained by saving Dataframes from text files
def create_xlsx_files(src: Path | str, dst: Path | str):
    # first read text files in src
    src = Path(src) # to make sure that src is actually Path
    # using Path objects lets us use glob below
    text_files = sorted(src.glob('*.txt'))

    # create dst if it does not exist
    dst = Path(dst)
    dst.mkdir(exist_ok=True)
    # now simply loop through text_files and create Dataframe and save in dst as text file with xlsx suffix
    for p in text_files:
        df = load_messy_file(p) # note this df is local to this function, outside df is not affected
        # df.to_excel(dst / (p.stem + ".xlsx"))
        # we want to save without index
        df.to_excel(dst / (p.stem + ".xlsx"), index=False)

### Saving converted files into XLSX

Now that we have needed functions we can use them to convert text files to XLSX format.

Of course the text files should match the specific format (have separator hint and naturally have lines of content separated by that separtor

In [80]:
# let'' take day_2_data text files
# and convert them to xlsx
# let's use new folder for that
# let's call it day_2_xlsx
INPUT_DIR = "day_2_data" # note ALL_CAPS is not required, it just shows that these are configurable variables
OUTPUT_DIR = "day_2_xlsx"

create_xlsx_files(INPUT_DIR, OUTPUT_DIR)

In [81]:
# let us download all files from OUTPU_DIR
from google.colab import files
for p in sorted(Path(OUTPUT_DIR).glob("*.xlsx")):
    files.download(p)
# TODO create function to download all files from some folder in colab to your local computer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 2: Guided Exercise — Latvia Weather Data (Extra Messy)

**Duration:** ~30 minutes  
**Dataset:** `latvia_meteo_1925_extra_messy.zip`  
**URL:** https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip

### 🎯 Objective
Convert multiple extra-messy weather text files   into XLSX files into a new folder of your choice

### Hint

Simply run the provided functions with folders of your choice.

### Advanced Users

Advanced users with some Pandas experience can work on checking for missing values and even try merging or concatenating the dataframes.

In [ ]:
# --- SKELETON (students fill in) ---
EXTRA_URL = 'https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip'
DATA_DIR = Path('day_2_exercise')

# 1) Download & unzip
download_and_unzip(EXTRA_URL, DATA_DIR)

# # 2) Print contents of DATA DIR

data_files = sorted(DATA_DIR.glob('*.txt'))
for file_name in data_files: # file_names is just a variable name
  print(file_name)

# 2) Inspect: list files & counts
for p in sorted(DATA_DIR.glob('*.txt')):
    print(p.name, '->', count_lines(p))



day_2_exercise/dobele_1925.txt
day_2_exercise/mersrags_extra_1925.txt
day_2_exercise/pavilosta_1925.txt
dobele_1925.txt -> 54
mersrags_extra_1925.txt -> 52
pavilosta_1925.txt -> 56


In [ ]:
# # let us test on day_2_exercise folder
INPUT_FOLDER = Path("day_2_exercise") # this should actually exist
OUTPUT_FOLDER = Path("day_2_exercise_output") # this can be any valid folder name
create_xlsx_files(INPUT_FOLDER, OUTPUT_FOLDER)

In [ ]:
# let us download all files from OUTPUT_FOLDER
# from google.colab import files # not necessary if you already imported before
for file in sorted(Path(OUTPUT_FOLDER).glob("*.xlsx")):
    files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# let us create a function that takes downloads files from Google Colab to local computer
# this function takes two arguments
# first argument is folder where files reside
# second argument is extension that files should match
def download_from_colab(folder=".", extension=".xlsx", subfolders=True, verbose=False):
  """
  Downloads files from Google Colab to the local computer.

  Args:
    folder (str or Path): The folder where files reside. Defaults to the current directory.
    extension (str): The file extension to match. Defaults to ".xlsx".
    subfolders (bool): Whether to search in subfolders. Defaults to True.
    verbose (bool): Whether to print information about the download process. Defaults to False.
  """
  if subfolders:
    files_to_download = sorted(Path(folder).rglob(f"*{extension}"))
  else:
    files_to_download = sorted(Path(folder).glob(f"*{extension}"))

  if verbose:
      print(f"Found {len(files_to_download)} files with extension {extension} in folder {folder}")

  for p in files_to_download:
    if verbose:
        print(f"Downloading {p}")
    files.download(p)

In [ ]:
# now I will call this function without any arguments so will use all defaults
download_from_colab()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# how about with verbose=True
download_from_colab(verbose=True)

### 🧪 Checkpoints
- At least **N≥3** cleaned files successfully load into DataFrames.
- No parsing exceptions on `.head()` or `.info()`.
- You can explain (in comments) which rules your `is_good_line` used.

### 🛠 Extension (Optional)
- Write a variant `clean_files(folder, out_dir=Path('data/cleaned'))` that writes outputs into a subfolder.
- Add a **regex-based** `is_good_line_regex` that only keeps lines starting with `YYYY-MM-DD`.

## Part 2b: Extracting column names automatically

We were able to extract the data from the files and load them into DataFrames and then save them into XLSX files.

One negative is that we do not have column names just numbers.

Thus we would like to extract column names automatically

## Part 3: Pandas-Specific Data Cleaning

Now we will focus on cleaning the data using pandas.



### Overview
In this section, you will standardize each DataFrame from Part 2 so they share a **common schema** and are ready to merge.

### Target Schema (example)
- `date` (datetime)
- `station` (string/category)
- `t_min` (float)
- `t_max` (float)
- `precip` (float)

### Typical Operations
1. **Column detection & renaming** – bring different column names to a shared set
2. **Type coercion** – numbers via `pd.to_numeric(errors='coerce')`, dates via `pd.to_datetime(errors='coerce')`
3. **Missing values** – `dropna` or `fillna` depending on context
4. **Duplicates** – `.duplicated()` + `.drop_duplicates()`
5. **Categoricals** – normalize text (`strip`, `title`, `upper`) and `astype('category')` if useful
6. **Validation** – quick assertions (e.g., date not null, temperature ranges plausible)

### Step-by-Step Guide
1) **Pick one DataFrame** from `dfs_extra` and print `.head()`, `.columns`, `.info()`
2) **Map columns** to target names (e.g., `temp_min` → `t_min`)
3) **Coerce**:
   - `date = pd.to_datetime(df['date'], errors='coerce')`
   - `df[['t_min','t_max','precip']] = df[['t_min','t_max','precip']].apply(pd.to_numeric, errors='coerce')`
4) **Handle missing**: start conservative (e.g., drop rows missing `date` or all temperature columns)
5) **Standardize station names**: `df['station'] = df['station'].astype(str).str.strip().str.title()`
6) **Check duplicates** and remove
7) **Repeat** for all DataFrames

### Common Pitfalls & Tips
- Treat ambiguous `-` or `NA` strings as missing (`na_values=["-","NA","N/A"]` if you re-read with `read_csv`)
- Some files might have **merged columns**; split using `.str.split(',', expand=True)` when necessary
- If a file lacks a column, create it with `pd.NA` so the schema lines up later

### 🧱 Skeleton: Inspect & Rename

In [ ]:
# let us load xlsx files currently available
# xlsx_files = sorted(Path("day_2_xlsx").glob("*.xlsx"))
# i want ALL XLSX files currently available in all subfolders
xlsx_files = sorted(Path(".").rglob("*.xlsx"))
print(f"Found {len(xlsx_files)} files")
dfs_extra = {} # i create a blank dictionary
# keys will be file name stems
# and values will be dataframes for that file name
for p in xlsx_files:
    dfs_extra[p.stem] = pd.read_excel(p)


Found 8 files


In [ ]:
## let's see what names we have for our keys
print(dfs_extra.keys())

dict_keys(['dobele_1925', 'mersrags_extra_1925', 'pavilosta_1925', 'aluksne_1925', 'liepaja_1925', 'mersrags_1925', 'riga_university_1925_p1', 'riga_university_1925_p2'])


In [ ]:
# how many key - value pairs?
print(len(dfs_extra.keys()))

8


In [ ]:
# let us look at first df in dfs_extra
df = dfs_extra['aluksne_1925'] # this is how I get value from key
df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5\n
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2\n
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5\n
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4\n
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—\n


In [ ]:
# let's look at liepaja_1925'
df = dfs_extra['liepaja_1925']
df.head()

,0,1,2,3,4,5,6,7,8,9
0,13,2,1925/05/26,none,75,0,0 mm,-999,Mērīšanas kļūda?\n,NaN
1,24,4,07.06.1925,rain,75,0,6,16,8,Neliels vējš\n
2,12,600000000000001,1925/05/20,S,50,0,4 mm,13,3,Pūtis brāzmas\n
3,15,8,1925/06/16,NaN,73,0,2,10,8,Skursteņi kūp\n
4,21,9,1925/06/25 07:00,—,81,0,0 mm,15,4,NA\n


In [ ]:
# let's remove liepaja_1925 from our dictionary
del dfs_extra['liepaja_1925'] # this drops the key-value pair
# Python Garbage collector will remove the liepaja_1925 dataframe from memory when there are no variables referrring to
# how many keys we have now? should have 7
print(len(dfs_extra.keys()))

7


In [ ]:
# let's check if all remaining dataframes have same columns (not necessarily in same order)
column_dictionary = {}
for k, v in dfs_extra.items():
    column_dictionary[k] = set(v.columns.tolist()) # set creates a collection of unique items from list or any other iterable
# keys
print(column_dictionary.keys())


dict_keys(['dobele_1925', 'mersrags_extra_1925', 'pavilosta_1925', 'aluksne_1925', 'mersrags_1925', 'riga_university_1925_p1', 'riga_university_1925_p2'])


In [ ]:
# let's set first value in column_dictionary as reference and check if we have any differences
unique_columns = column_dictionary['dobele_1925']
print(f"Will compare against following columns: {unique_columns}")
for k, v in column_dictionary.items():
    # print differences for each value set with unique_columns
    print(k, unique_columns.symmetric_difference(v)) # so we see what if any columns are different with reference set of columns

Will compare against following columns: {'precip_24h_mm', 'precip_type', 't_min_c', 'date', 'notes', 't_max_c', 'present_weather_code'}
dobele_1925 set()
mersrags_extra_1925 set()
pavilosta_1925 set()
aluksne_1925 set()
mersrags_1925 set()
riga_university_1925_p1 set()
riga_university_1925_p2 set()


In [ ]:
# so what are those unique_columns
# let's see them sorted
print(sorted(unique_columns))

['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code', 't_max_c', 't_min_c']


In [ ]:
aluksne_df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—


In [ ]:
aluksne_df['veikals'] = "RIMI" # all cells in column veikals will have value RIMI
aluksne_df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c,veikals
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5,RIMI
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2,RIMI
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5,RIMI
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4,RIMI
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—,RIMI


In [ ]:
# so now let's drop this column
aluksne_df = aluksne_df.drop(columns=['veikals'])
aluksne_df.head()

,notes,present_weather_code,t_max_c,precip_24h_mm,date,precip_type,t_min_c
0,—,53.0,5.9,3.0 mm,10-19-1925,R,5.5
1,Rīts auksts,-999.0,9.1,2.3 mm,18.10.1925,mixed,4.2
2,Daļēji mākoņains,65.0,5.4,0.0,11-10-1925,NaN,-0.5
3,Daļēji mākoņains,50.0,15.6,0.5,10-07-1925 14:00,S,10.4
4,Neliels vējš,63.0,-1.1,0.0,12-28-1925,—,—


In [ ]:
# firs let's drop the 'Unnamed: 0' column from all dataframes in our dfs_extra dataframes
column_to_drop = 'Unnamed: 0'
for k, v in dfs_extra.items(): # so k is file stem, and v is the actual dataframe
    print(f"Dropping {column_to_drop} from {k} dataframe shaped {dfs_extra[k].shape}")
    if column_to_drop in dfs_extra[k].columns: # before I "shoot" I check whether there is anything worth shooting
        dfs_extra[k] = dfs_extra[k].drop(columns=[column_to_drop])
        # shape after dropping
        print(f"Shape after dropping: {dfs_extra[k].shape}")
    else:
        print(f"No need to remove {column_to_drop} as it does not exist")

Dropping Unnamed: 0 from dobele_1925 dataframe shaped (48, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from mersrags_extra_1925 dataframe shaped (46, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from pavilosta_1925 dataframe shaped (50, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from aluksne_1925 dataframe shaped (36, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from mersrags_1925 dataframe shaped (48, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from riga_university_1925_p1 dataframe shaped (36, 7)
No need to remove Unnamed: 0 as it does not exist
Dropping Unnamed: 0 from riga_university_1925_p2 dataframe shaped (36, 7)
No need to remove Unnamed: 0 as it does not exist


In [ ]:
# so let's sort the unique columns again and save them in a list
unique_columns = sorted(unique_columns)
# let's see them
print(unique_columns)

['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code', 't_max_c', 't_min_c']


In [ ]:
# drop the first entry
# unique_columns.pop(0) # this pops from beginning (Note in very large lists this can be slow)
# show again
print(unique_columns)

['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code', 't_max_c', 't_min_c']


In [ ]:
# so let's use unique_columns order for all dataframes
for k, v in dfs_extra.items():
    dfs_extra[k] = v[unique_columns] # in order for this to work the columns should exist
    # print columns
    print(k, dfs_extra[k].columns)


dobele_1925 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
mersrags_extra_1925 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
pavilosta_1925 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
aluksne_1925 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
mersrags_1925 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
riga_university_1925_p1 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
       't_max_c', 't_min_c'],
      dtype='object')
riga_university_1925_p2 Index(['date', 'notes', 'precip_24h_mm', 'precip_type', 'present_weather_code',
      

In [ ]:
# now we can concatante all 7 dataframes in one big dataframe
# first we want to add column file_name using key for all dataframes so we do not lose reference later
for k, v in dfs_extra.items():
    dfs_extra[k]['file_name'] = k # similar to how we assigned same value of "RIMI" to our store column previous day
    # show first 3 rows
    display(dfs_extra[k].head(3))



,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,22-08-1925,Laikam migla,"1,2",rain\n,75.0,"16,0","8,5",dobele_1925
1,1925/08/03,Skursteņi kūp,"3,5",snow\n,60.0,"18,5","19,2",dobele_1925
2,24-08-1925 14:00,Pūtis brāzmas,"0,0 mm",snow\n,50.0,"17,6",NaN,dobele_1925


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,1925.09.13,Skursteņi kūp,\n,R,82.0,15.5,13.7,mersrags_extra_1925
1,23-08-1925,Pūtis brāzmas,0.0\n,NaN,NaN,NaN,16.7,mersrags_extra_1925
2,07/07/1925,NaN,0.0\n,none,65.0,17.7,18.4,mersrags_extra_1925


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,1925.06.06 14:00,—\n,0.0,NaN,80.0,24.2,NaN,pavilosta_1925
1,1925.05.20 14:00,Daļēji mākoņains\n,0.2 mm,—,NaN,5.8,6.5,pavilosta_1925
2,1925.03.17,Pūtis brāzmas\n,0.8,rain,73.0,1.4,-1.1,pavilosta_1925


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,10-19-1925,—,3.0 mm,R,53.0,5.9,5.5\n,aluksne_1925
1,18.10.1925,Rīts auksts,2.3 mm,mixed,-999.0,9.1,4.2\n,aluksne_1925
2,11-10-1925,Daļēji mākoņains,0.0,NaN,65.0,5.4,-0.5\n,aluksne_1925


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,15-03-1925,Mērījums apstiprināts,0.0,none,70,-4.4\n,-3.7,mersrags_1925
1,04-04-1925 14:00,Ap pusdienlaiku saule,0.6,snow,75,7.9\n,4.7,mersrags_1925
2,22-05-1925,NaN,0.1,NaN,80,11.9\n,7.7,mersrags_1925


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,1925-02-11,—\n,0.0,NaN,73.0,1.3,—,riga_university_1925_p1
1,1925-01-06,Mērīšanas kļūda?\n,0.1 mm,none,90.0,0.8,-4.7,riga_university_1925_p1
2,1925-02-07,Rīts auksts\n,0.0 mm,none,71.0,-7.2,-8.3,riga_university_1925_p1


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,18.10.1925 20:00,—\n,0.7,snow,53,10.5,6.0,riga_university_1925_p2
1,1925/10/13,\n,0.0,—,NaN,9.7,8.1,riga_university_1925_p2
2,16.09.1925,Mērīšanas kļūda?\n,1.3,rain,50,17.3,14.8,riga_university_1925_p2


In [ ]:
# finally we can concatanate all the dataframes into one big dataframe
df = pd.concat(dfs_extra.values(), ignore_index=True) # values are the dataframes, we do not need keys(file_names) any more
# shape after concatenation
print(f"Shape after concatenation: {df.shape}")
# random sample
df.sample(5, random_state=2025)

Shape after concatenation: (300, 8)


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
102,1925.06.22,Novērojums vēlāk\n,nulle,snow,71.0,17.1,12.2,pavilosta_1925
148,12-28-1925,Neliels vējš,0.0,—,63.0,-1.1,—\n,aluksne_1925
175,12-18-1925,Mērījums apstiprināts,0.2 mm,snow,51.0,-2.2,-7.6\n,aluksne_1925
183,13-03-1925,Skursteņi kūp,0.0,NaN,—,2.8\n,3.5,mersrags_1925
225,12-04-1925 13:00,Novērojums vēlāk,0.0,NaN,45,8.5\n,1.4,mersrags_1925


In [ ]:
# let's add location column by splitting file_name by _ and taking first part
df['location'] = df.file_name.str.split("_").str[0] # Corrected to apply split and index to each element
# sample again
df.sample(5, random_state=2025)

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name,location
102,1925.06.22,Novērojums vēlāk\n,nulle,snow,71.0,17.1,12.2,pavilosta_1925,pavilosta
148,12-28-1925,Neliels vējš,0.0,—,63.0,-1.1,—\n,aluksne_1925,aluksne
175,12-18-1925,Mērījums apstiprināts,0.2 mm,snow,51.0,-2.2,-7.6\n,aluksne_1925,aluksne
183,13-03-1925,Skursteņi kūp,0.0,NaN,—,2.8\n,3.5,mersrags_1925,mersrags
225,12-04-1925 13:00,Novērojums vēlāk,0.0,NaN,45,8.5\n,1.4,mersrags_1925,mersrags


In [ ]:
# let's save the big dataframe without index
df.to_excel("day_2_xlsx/all_stations_without_liepaja_cleaned.xlsx", index=False)


In [ ]:
# now let's download to local computer
from google.colab import files
files.download("day_2_xlsx/all_stations_without_liepaja_cleaned.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loading All Stations From Excel

URL - https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/all_stations_without_liepaja_cleaned.xlsx

In [ ]:
all_stations_url = "https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/all_stations_without_liepaja_cleaned.xlsx"
print(f"Downloading {all_stations_url}")
df = pd.read_excel(all_stations_url)
print(f"Shape after loading all stations: {df.shape}")
df.head()

Shape after loading all stations: (300, 8)


,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,22-08-1925,Laikam migla,"1,2",rain\n,75,"16,0","8,5",dobele_1925
1,1925/08/03,Skursteņi kūp,"3,5",snow\n,60,"18,5","19,2",dobele_1925
2,24-08-1925 14:00,Pūtis brāzmas,"0,0 mm",snow\n,50,"17,6",NaN,dobele_1925
3,1925/05/08,Novērojums vēlāk,"0,0",none\n,81,"17,7","5,9",dobele_1925
4,11-07-1925 07:00,Neliels vējš,"0,3",mixed\n,51,"19,9","16,1",dobele_1925


In [ ]:
# assert False # we want to stop here
# we do this if we want Run All to stop here

AssertionError: 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  300 non-null    object
 1   notes                 262 non-null    object
 2   precip_24h_mm         283 non-null    object
 3   precip_type           241 non-null    object
 4   present_weather_code  271 non-null    object
 5   t_max_c               277 non-null    object
 6   t_min_c               282 non-null    object
 7   file_name             300 non-null    object
dtypes: object(8)
memory usage: 18.9+ KB


## Goals For Type conversion

We want to convert as many columns as possible to correct data types
such as t max and t min into floats and datae into special datetime index

In [ ]:
# let us strip all alpha characters from precip_24h_mm column
df['precip_24h_mm'] = df['precip_24h_mm'].str.strip('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ') # we could add other characters like Latvian characters
df.head()

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,22-08-1925,Laikam migla,"1,2",rain\n,75,"16,0","8,5",dobele_1925
1,1925/08/03,Skursteņi kūp,"3,5",snow\n,60,"18,5","19,2",dobele_1925
2,24-08-1925 14:00,Pūtis brāzmas,"0,0",snow\n,50,"17,6",NaN,dobele_1925
3,1925/05/08,Novērojums vēlāk,"0,0",none\n,81,"17,7","5,9",dobele_1925
4,11-07-1925 07:00,Neliels vējš,"0,3",mixed\n,51,"19,9","16,1",dobele_1925


In [ ]:
# we see that some entries in precip_24h_mm are using , instead of .
# let's convert that
df['precip_24h_mm'] = df['precip_24h_mm'].str.replace(',', '.')
# if i was unsure I could have created a new column and then checked for conversion
df.head()

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,22-08-1925,Laikam migla,1.2,rain\n,75,"16,0","8,5",dobele_1925
1,1925/08/03,Skursteņi kūp,3.5,snow\n,60,"18,5","19,2",dobele_1925
2,24-08-1925 14:00,Pūtis brāzmas,0.0,snow\n,50,"17,6",NaN,dobele_1925
3,1925/05/08,Novērojums vēlāk,0.0,none\n,81,"17,7","5,9",dobele_1925
4,11-07-1925 07:00,Neliels vējš,0.3,mixed\n,51,"19,9","16,1",dobele_1925


In [ ]:
# let's do same for t_max_c and t_min_c
# this means that any not a number mentions that are not standard will end up with blank strings ""
df['t_max_c'] = df['t_max_c'].str.strip('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
df['t_max_c'] = df['t_max_c'].str.replace(',', '.')

df['t_min_c'] = df['t_min_c'].str.strip('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
df['t_min_c'] = df['t_min_c'].str.replace(',', '.')
df.head()

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name
0,22-08-1925,Laikam migla,1.2,rain\n,75,16.0,8.5,dobele_1925
1,1925/08/03,Skursteņi kūp,3.5,snow\n,60,18.5,19.2,dobele_1925
2,24-08-1925 14:00,Pūtis brāzmas,0.0,snow\n,50,17.6,NaN,dobele_1925
3,1925/05/08,Novērojums vēlāk,0.0,none\n,81,17.7,5.9,dobele_1925
4,11-07-1925 07:00,Neliels vējš,0.3,mixed\n,51,19.9,16.1,dobele_1925


In [ ]:
# let us check data types again
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  300 non-null    object
 1   notes                 262 non-null    object
 2   precip_24h_mm         283 non-null    object
 3   precip_type           241 non-null    object
 4   present_weather_code  271 non-null    object
 5   t_max_c               135 non-null    object
 6   t_min_c               239 non-null    object
 7   file_name             300 non-null    object
dtypes: object(8)
memory usage: 18.9+ KB


In [ ]:
# let's convert precip_24h_mm, t_max_c,	t_min_c columns to numeric
df['precip_24h_mm'] = pd.to_numeric(df['precip_24h_mm'], errors='coerce')
df['t_max_c'] = pd.to_numeric(df['t_max_c'], errors='coerce')
df['t_min_c'] = pd.to_numeric(df['t_min_c'], errors='coerce')
# without coerce the conversion might fail, then we keep cleaning until we can convert

# if we are worried we might lose some potential candidation we could save into new or different columns


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  300 non-null    object 
 1   notes                 262 non-null    object 
 2   precip_24h_mm         245 non-null    float64
 3   precip_type           241 non-null    object 
 4   present_weather_code  271 non-null    object 
 5   t_max_c               126 non-null    float64
 6   t_min_c               230 non-null    float64
 7   file_name             300 non-null    object 
dtypes: float64(3), object(5)
memory usage: 18.9+ KB


In [ ]:
# let us convert date column to date type
# in this case I keep original date column because our conversion might fail many times and generate Nan
df['date_converted'] = pd.to_datetime(df['date'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  300 non-null    object        
 1   notes                 262 non-null    object        
 2   precip_24h_mm         245 non-null    float64       
 3   precip_type           241 non-null    object        
 4   present_weather_code  271 non-null    object        
 5   t_max_c               126 non-null    float64       
 6   t_min_c               230 non-null    float64       
 7   file_name             300 non-null    object        
 8   date_converted        77 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 21.2+ KB


/tmp/ipython-input-1447983913.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date_converted'] = pd.to_datetime(df['date'], errors='coerce')


In [ ]:
# let us count how many nan values date_converted has
df['date_converted'].isna().sum()

np.int64(223)

In [ ]:
# let us see all date cells for rows where date_converted is nan
df[df['date_converted'].isna()][['date', 'date_converted']] # again note double brackets for showing more than one column

,date,date_converted
1,1925/08/03,NaT
2,24-08-1925 14:00,NaT
3,1925/05/08,NaT
4,11-07-1925 07:00,NaT
9,06-15-1925,NaT
...,...,...
295,02.10.1925 19:00,NaT
296,12.10.1925,NaT
297,17.11.1925 07:00,NaT
298,15.09.1925,NaT


In [ ]:
# convert cells in date column which match DD.MM.YYYY format to date_converted column
print(f"{df['date_converted'].isna().sum()} nan values left in date_converted")
df.loc[df['date'].str.match(r'\d{2}\.\d{2}\.\d{4}'), 'date_converted'] = pd.to_datetime(df[df['date'].str.match(r'\d{2}\.\d{2}\.\d{4}')]['date'], errors='coerce')
# note we use raw strings r"this is raw string" which does not use escaping
# why? because regular expressions also use escaping
# how many nan values  left in date_converted
print(f"{df['date_converted'].isna().sum()} nan values left in date_converted")
# we saved a trip to regex101.com here by using AI assitance

# TODO at home convert the rest of datatime using regex for each different case

223 nan values left in date_converted
210 nan values left in date_converted


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  300 non-null    object        
 1   notes                 262 non-null    object        
 2   precip_24h_mm         245 non-null    float64       
 3   precip_type           241 non-null    object        
 4   present_weather_code  271 non-null    object        
 5   t_max_c               126 non-null    float64       
 6   t_min_c               230 non-null    float64       
 7   file_name             300 non-null    object        
 8   date_converted        90 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 21.2+ KB


In [ ]:
# let us see all date cells for rows where date_converted is nan
df[df['date_converted'].isna()][['date', 'date_converted']]

,date,date_converted
1,1925/08/03,NaT
2,24-08-1925 14:00,NaT
3,1925/05/08,NaT
4,11-07-1925 07:00,NaT
9,06-15-1925,NaT
...,...,...
291,14.09.1925,NaT
294,20.09.1925 20:00,NaT
295,02.10.1925 19:00,NaT
297,17.11.1925 07:00,NaT


In [ ]:
# let us convert all dates in date column that look like 1925/08/03 to correct datetime in date_converted column
print(f"{df['date_converted'].isna().sum()} nan values left in date_converted")
df.loc[df['date'].str.match(r'\d{4}/\d{2}/\d{2}'), 'date_converted'] = pd.to_datetime(df[df['date'].str.match(r'\d{4}/\d{2}/\d{2}')]['date'], errors='coerce')
# note we use raw strings r"this is raw string" which does not use escaping
# why? because regular expressions also use escaping
# how many nan values  left in date_converted
print(f"{df['date_converted'].isna().sum()} nan values left in date_converted")


210 nan values left in date_converted
203 nan values left in date_converted


In [ ]:
# now we can do describe
df.describe()

,precip_24h_mm,t_max_c,t_min_c,date_converted
count,245.000000,126.000000,230.000000,97
mean,-23.619184,-19.555556,-29.070435,1925-06-08 22:01:14.226804224
min,-999.000000,-999.000000,-999.000000,1925-01-11 00:00:00
25%,0.000000,5.225000,-1.900000,1925-04-08 00:00:00
50%,0.400000,12.950000,6.500000,1925-05-26 00:00:00
75%,1.300000,18.500000,12.275000,1925-08-06 00:00:00
max,6.000000,26.100000,22.300000,1925-12-10 00:00:00
std,154.863589,178.211225,184.697402,NaN


In [ ]:
# so -999 is an obvious outlier, it actually represents Nan so lets convert all -999 occurences to Nan
df.loc[df['t_min_c'] == -999, 't_min_c'] = pd.NA # pd.NA
df.describe()

,precip_24h_mm,t_max_c,t_min_c,date_converted
count,245.000000,126.000000,222.000000,97
mean,-23.619184,-19.555556,5.881982,1925-06-08 22:01:14.226804224
min,-999.000000,-999.000000,-11.800000,1925-01-11 00:00:00
25%,0.000000,5.225000,-1.300000,1925-04-08 00:00:00
50%,0.400000,12.950000,6.850000,1925-05-26 00:00:00
75%,1.300000,18.500000,13.000000,1925-08-06 00:00:00
max,6.000000,26.100000,22.300000,1925-12-10 00:00:00
std,154.863589,178.211225,8.124848,NaN


In [ ]:
# let's convert precip_24h_mm	t_max_c -999 also to NA
df.loc[df['precip_24h_mm'] == -999, 'precip_24h_mm'] = pd.NA
df.loc[df['t_max_c'] == -999, 't_max_c'] = pd.NA
df.describe()

,precip_24h_mm,t_max_c,t_min_c,date_converted
count,239.000000,122.000000,222.000000,97
mean,0.867364,12.557377,5.881982,1925-06-08 22:01:14.226804224
min,0.000000,-4.400000,-11.800000,1925-01-11 00:00:00
25%,0.000000,6.050000,-1.300000,1925-04-08 00:00:00
50%,0.500000,13.500000,6.850000,1925-05-26 00:00:00
75%,1.400000,18.575000,13.000000,1925-08-06 00:00:00
max,6.000000,26.100000,22.300000,1925-12-10 00:00:00
std,1.080277,7.523000,8.124848,NaN


In [ ]:
# let us do the same replacement of -999 to NaN for present_weather_code column
df.loc[df['present_weather_code'] == -999, 'present_weather_code'] = pd.NA
df.describe()

# how to understand the above operation
# we look for rows where present_weather_code is -999
# in those rows we change value of well present_weather_code to Not a Number or NaN

,precip_24h_mm,t_max_c,t_min_c,date_converted
count,239.000000,122.000000,222.000000,97
mean,0.867364,12.557377,5.881982,1925-06-08 22:01:14.226804224
min,0.000000,-4.400000,-11.800000,1925-01-11 00:00:00
25%,0.000000,6.050000,-1.300000,1925-04-08 00:00:00
50%,0.500000,13.500000,6.850000,1925-05-26 00:00:00
75%,1.400000,18.575000,13.000000,1925-08-06 00:00:00
max,6.000000,26.100000,22.300000,1925-12-10 00:00:00
std,1.080277,7.523000,8.124848,NaN


In [ ]:
# let us see head again
df.head()

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name,date_converted
0,22-08-1925,Laikam migla,1.2,rain\n,75,16.0,8.5,dobele_1925,1925-08-22
1,1925/08/03,Skursteņi kūp,3.5,snow\n,60,18.5,19.2,dobele_1925,1925-08-03
2,24-08-1925 14:00,Pūtis brāzmas,0.0,snow\n,50,17.6,NaN,dobele_1925,NaT
3,1925/05/08,Novērojums vēlāk,0.0,none\n,81,17.7,5.9,dobele_1925,1925-05-08
4,11-07-1925 07:00,Neliels vējš,0.3,mixed\n,51,19.9,16.1,dobele_1925,NaT


In [ ]:
# let us create a duplicate of first row at the very end
df.loc[len(df)] = df.iloc[0]
df.tail()

,date,notes,precip_24h_mm,precip_type,present_weather_code,t_max_c,t_min_c,file_name,date_converted
296,12.10.1925,\n,2.0,R,NaN,NaN,1.0,riga_university_1925_p2,1925-12-10
297,17.11.1925 07:00,Novērojums vēlāk\n,0.0,NaN,80,NaN,1.0,riga_university_1925_p2,NaT
298,15.09.1925,Vēlāk sāka līt\n,0.7,mixed,81,NaN,3.9,riga_university_1925_p2,NaT
299,08.11.1925,\n,NaN,NaN,71,NaN,-1.9,riga_university_1925_p2,1925-08-11
300,22-08-1925,Laikam migla,1.2,rain\n,75,16.0,8.5,dobele_1925,1925-08-22


In [ ]:
# so let's see shape before we drop duplicates
print(f"Shape before dropping duplicates: {df.shape}")
# let us drop duplicates
df = df.drop_duplicates()
# drop_duplicates also has subset parameter which you can use to pass specific columns to consider for duplicate status
# default is to only drop when there is 100% match in ALL columns
print(f"Shape after dropping duplicates: {df.shape}")

Shape before dropping duplicates: (301, 9)
Shape after dropping duplicates: (300, 9)


In [ ]:
# let us save to xlxs WITHOUT index
# first let's create day_2_xlsx folder IF it does not exist using Path
Path("day_2_xlsx").mkdir(exist_ok=True)
# now let's save

df.to_excel("day_2_xlsx/all_stations_without_Liepaja_cleaned_and_partially_converted.xlsx", index=False)

In [ ]:
# let us download to our own computer
from google.colab import files
files.download("day_2_xlsx/all_stations_without_Liepaja_cleaned_and_partially_converted.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Next steps

So we learned to manipulate and clean columns in various ways

* change column order for ALL dataframes to be merged
* combine dataframes with merge or concat commands


In [ ]:
# Example skeleton for one dataframe named df
# df = dfs_extra['some_file']
# print(df.head()); print(df.columns); df.info()

# rename_map = {
#     'Date': 'date', 'DATE':'date',
#     'Station':'station', 'City':'station',
#     'Tmin':'t_min', 'TminC':'t_min', 'Min':'t_min',
#     'Tmax':'t_max', 'TmaxC':'t_max', 'Max':'t_max',
#     'Precip':'precip', 'Rain':'precip'
# }
# df = df.rename(columns=lambda c: rename_map.get(str(c), str(c).strip().lower()))


### 🧱 Skeleton: Type Coercion & Missing Handling

In [ ]:
# required_cols = ['date','station','t_min','t_max','precip']
# for c in required_cols:
#     if c not in df.columns:
#         df[c] = pd.NA

# df['date'] = pd.to_datetime(df['date'], errors='coerce')
# for c in ['t_min','t_max','precip']:
#     df[c] = pd.to_numeric(df[c], errors='coerce')

# # Drop rows with no usable date
# df = df.dropna(subset=['date'])

# # Optional: fill precip missing with 0 if domain-appropriate
# # df['precip'] = df['precip'].fillna(0)

### 🧱 Skeleton: Text Normalization & Duplicates

In [ ]:
# df['station'] = df['station'].astype(str).str.strip().str.title()
# before = len(df)
# df = df.drop_duplicates()
# print('Removed', before - len(df), 'duplicate rows')

### 🧪 Suggested Sanity Checks

In [ ]:
# assert df['date'].notna().all(), 'Null dates remain'
# # Optional plausibility checks (adjust to real units)
# assert (df['t_min'] <= df['t_max']).dropna().all(), 'Found t_min > t_max'

## Part 4: Merging Cleaned DataFrames

### Goal
Combine all standardized DataFrames into **one big DataFrame** with a **unified column structure**.

### Strategy
1. **Define the target schema** used in Part 3.
2. **Align each DataFrame** to the schema (add missing columns, reorder).
3. **Concatenate** with `pd.concat`.
4. **Final cleanup**: deduplicate, reindex, and sort by date/station.
5. **Save outputs** (`CSV` or `Parquet`) for Day 3 (EDA).

### Integration Checklist
- All DataFrames have columns: `date, station, t_min, t_max, precip`
- Dtypes are consistent across DataFrames
- No catastrophic loss of rows during coercion
- Final row count equals the sum of inputs minus duplicates

### 🧱 Skeleton: Alignment & Concatenation

In [ ]:
# Suppose you have a dict of cleaned dfs: dfs_clean
# target_cols = ['date','station','t_min','t_max','precip']

# def coerce_to_schema(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
#     for c in cols:
#         if c not in df.columns:
#             df[c] = pd.NA
#     # Reorder and drop extras for now
#     return df[cols]

# aligned = [coerce_to_schema(d.copy(), target_cols) for d in dfs_clean.values()]
# big = pd.concat(aligned, axis=0, ignore_index=True)
# big = big.drop_duplicates().reset_index(drop=True)
# big = big.sort_values(['date','station'])
# big.head()

### 🧾 Export for Day 3

In [ ]:
# out_dir = Path('outputs'); out_dir.mkdir(exist_ok=True)
# big.to_csv(out_dir / 'latvia_meteo_1925_cleaned_merged.csv', index=False)
# # Optional: Parquet for speed/size
# # big.to_parquet(out_dir / 'latvia_meteo_1925_cleaned_merged.parquet', index=False)

## 🔄 Reflection
- What kinds of messiness were easier to fix with **Python basics**?
- What kinds of messiness required **pandas**?
- What are the risks of “over-cleaning” or discarding too much data?